In [20]:
import os
import pandas as pd
import numpy as np
import tqdm
import pydicom

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [29]:
df = pd.read_excel('Ped_liver_QUS.xlsx')
print(df.shape)

(624, 19)


In [3]:
paths=df['directory']
fnames= df['rfFilename']

In [4]:
df['fullname'] = paths + "/" + fnames
print(df['fullname'])

0      /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
1      /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
2      /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
3      /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
4      /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
5      /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
6      /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
7      /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
8      /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
9      /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
10     /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
11     /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
12     /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
13     /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
14     /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
15     /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
16     /data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...
17     /data/QUS/UCSDLiver/Peds

In [5]:
full_path = df['fullname'].to_list()
full_path

['/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image1/IM-0001-0120_RF',
 '/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image1/IM-0001-0120_RF',
 '/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image2/IM-0001-0130_RF',
 '/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image2/IM-0001-0130_RF',
 '/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image3/IM-0001-0140_RF',
 '/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image3/IM-0001-0140_RF',
 '/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image4/IM-0001-0150_RF',
 '/data/QUS/UCSDLive

In [13]:
files_list= [s.replace('_RF', '.dcm') for s in full_path]
files_list= [s.replace('/data/QUS/UCSDLiver/Peds_Pilot_Data/', '') for s in files_list]
files_list

['UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image1/IM-0001-0120.dcm',
 'UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image1/IM-0001-0120.dcm',
 'UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image2/IM-0001-0130.dcm',
 'UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image2/IM-0001-0130.dcm',
 'UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image3/IM-0001-0140.dcm',
 'UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image3/IM-0001-0140.dcm',
 'UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image4/IM-0001-0150.dcm',
 'UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image4/IM-0001-0150.dcm',
 'UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON/RFLiverAPhantomA/Liver/Image5/IM-0001-0160.dcm',
 'UIUC_Clinical_Packages/UP2101claude/Exam1/HarmonicsON

In [21]:
def save_value(img_data, name, value):
    if type(value) == pydicom.multival.MultiValue:
        for i, j in enumerate(value):
            save_value(img_data, f"{name}_{i}", j)
    else:
        if type(value) == pydicom.uid.UID:
            value = str(value)
        elif type(value) == pydicom.valuerep.DSfloat:
            value = float(value)
        img_data[name] = value

def get_data_dict(img):
    img_data = {}
    for i in img.iterall():
        if i.name == "Pixel Data":
            continue
        name = i.name.replace(" ", "_").replace("(", "").replace(")", "").lower()
        save_value(img_data, name, i.value)
    return img_data

def get_list_data(imgs):
    list_data = []
    for i in tqdm.tqdm(imgs):
        img = pydicom.read_file(i)
        img_data = get_data_dict(img)
        list_data.append(img_data)
    return list_data

def get_df_data(imgs):
    list_data = get_list_data(imgs)
    return pd.DataFrame(list_data)



In [26]:
df_imgs = get_df_data(files_list)
df_imgs.head()

100%|████████████████████████████████████████████████████████████████████████████████| 624/624 [02:15<00:00,  4.59it/s]


,image_type_0,image_type_1,image_type_2,image_type_3,image_type_4,image_type_5,sop_class_uid,sop_instance_uid,study_date,series_date,content_date,study_time,series_time,content_time,accession_number,modality,manufacturer,institution_name,referring_physician's_name,station_name,operators'_name,manufacturer's_model_name,patient's_name,patient_id,patient's_birth_date,patient's_birth_time,patient's_sex,patient's_age,device_serial_number,software_versions,heart_rate,transducer_data_0,transducer_data_1,transducer_data_2,sequence_of_ultrasound_regions,region_spatial_format,region_data_type,region_flags,region_location_min_x0,region_location_min_y0,region_location_max_x1,region_location_max_y1,reference_pixel_x0,reference_pixel_y0,physical_units_x_direction,physical_units_y_direction,reference_pixel_physical_value_x,reference_pixel_physical_value_y,physical_delta_x,physical_delta_y,transducer_frequency,transducer_type,study_instance_uid,series_instance_uid,study_id,series_number,instance_number,patient_orientation,samples_per_pixel,photometric_interpretation,planar_configuration,rows,columns,ultrasound_color_data_present,bits_allocated,bits_stored,high_bit,pixel_representation,lossy_image_compression,private_creator,private_tag_data,[unknown],derivation_description,study_description
0,DERIVED,SECONDARY,ABDOMINAL,0001,GEMSSINGLEFRAME,GEMSMGCOUNT1,1.2.840.10008.5.1.4.1.1.6.1,1.2.840.113619.2.455.140911794567.1637692529.15,20211123,20211123,20211123,122455,122455,123529,,US,GE Healthcare,LA JOLLA CAMPUS,(),ACUS502330US7,"(A, D, M)",LOGIQE10,(),UP2101_US_1,,000000,O,999Y,502330US7,LOGIQE10:R2.5.3,-1,C1-6,574,1373257431,"[[(0018, 6012) Region Spatial Format ...",1,1,0,2,109,1057,805,527,18,3,3,0.0,0.0,0.026664,0.026664,4500,CURVED LINEAR,1.2.840.113619.2.455.140911794567.1637691895.2,1.2.840.113619.2.455.140911794567.1637691895.3,20211123.122455,1,120,,3,RGB,0,248,264,0,8,8,7,0,00,GEMS_Ultrasound_MovieGroup_001,[],IQ,SmallPreview,NaN
1,DERIVED,SECONDARY,ABDOMINAL,0001,GEMSSINGLEFRAME,GEMSMGCOUNT1,1.2.840.10008.5.1.4.1.1.6.1,1.2.840.113619.2.455.140911794567.1637692529.15,20211123,20211123,20211123,122455,122455,123529,,US,GE Healthcare,LA JOLLA CAMPUS,(),ACUS502330US7,"(A, D, M)",LOGIQE10,(),UP2101_US_1,,000000,O,999Y,502330US7,LOGIQE10:R2.5.3,-1,C1-6,574,1373257431,"[[(0018, 6012) Region Spatial Format ...",1,1,0,2,109,1057,805,527,18,3,3,0.0,0.0,0.026664,0.026664,4500,CURVED LINEAR,1.2.840.113619.2.455.140911794567.1637691895.2,1.2.840.113619.2.455.140911794567.1637691895.3,20211123.122455,1,120,,3,RGB,0,248,264,0,8,8,7,0,00,GEMS_Ultrasound_MovieGroup_001,[],IQ,SmallPreview,NaN
2,DERIVED,SECONDARY,ABDOMINAL,0001,GEMSSINGLEFRAME,GEMSMGCOUNT1,1.2.840.10008.5.1.4.1.1.6.1,1.2.840.113619.2.455.140911794567.1637692532.16,20211123,20211123,20211123,122455,122455,123532,,US,GE Healthcare,LA JOLLA CAMPUS,(),ACUS502330US7,"(A, D, M)",LOGIQE10,(),UP2101_US_1,,000000,O,999Y,502330US7,LOGIQE10:R2.5.3,-1,C1-6,574,1373257431,"[[(0018, 6012) Region Spatial Format ...",1,1,0,2,109,1057,805,527,18,3,3,0.0,0.0,0.026664,0.026664,4500,CURVED LINEAR,1.2.840.113619.2.455.140911794567.1637691895.2,1.2.840.113619.2.455.140911794567.1637691895.3,20211123.122455,1,130,,3,RGB,0,248,264,0,8,8,7,0,00,GEMS_Ultrasound_MovieGroup_001,[],IQ,SmallPreview,NaN
3,DERIVED,SECONDARY,ABDOMINAL,0001,GEMSSINGLEFRAME,GEMSMGCOUNT1,1.2.840.10008.5.1.4.1.1.6.1,1.2.840.113619.2.455.140911794567.1637692532.16,20211123,20211123,20211123,122455,122455,123532,,US,GE Healthcare,LA JOLLA CAMPUS,(),ACUS502330US7,"(A, D, M)",LOGIQE10,(),UP2101_US_1,,000000,O,999Y,502330US7,LOGIQE10:R2.5.3,-1,C1-6,574,1373257431,"[[(0018, 6012) Region Spatial Format ...",1,1,0,2,109,1057,805,527,18,3,3,0.0,0.0,0.026664,0.026664,4500,CURVED LINEAR,1.2.840.113619.2.455.140911794567.1637691895.2,1.2.840.113619.2.455.140911794567.1637691895.3,20211123.122455,1,130,,3,RGB,0,248,264,0,8,8,7,0,00,GEMS_Ultrasound_MovieGroup_001,[],IQ,SmallPreview,NaN
4,DERIVED,SECONDARY,ABDOMINAL,0001,GEMSSINGLEFRAME,GEMSMGCOUNT1,1.2.

In [27]:
df_imgs.head(100)

,image_type_0,image_type_1,image_type_2,image_type_3,image_type_4,image_type_5,sop_class_uid,sop_instance_uid,study_date,series_date,content_date,study_time,series_time,content_time,accession_number,modality,manufacturer,institution_name,referring_physician's_name,station_name,operators'_name,manufacturer's_model_name,patient's_name,patient_id,patient's_birth_date,patient's_birth_time,patient's_sex,patient's_age,device_serial_number,software_versions,heart_rate,transducer_data_0,transducer_data_1,transducer_data_2,sequence_of_ultrasound_regions,region_spatial_format,region_data_type,region_flags,region_location_min_x0,region_location_min_y0,region_location_max_x1,region_location_max_y1,reference_pixel_x0,reference_pixel_y0,physical_units_x_direction,physical_units_y_direction,reference_pixel_physical_value_x,reference_pixel_physical_value_y,physical_delta_x,physical_delta_y,transducer_frequency,transducer_type,study_instance_uid,series_instance_uid,study_id,series_number,instance_number,patient_orientation,samples_per_pixel,photometric_interpretation,planar_configuration,rows,columns,ultrasound_color_data_present,bits_allocated,bits_stored,high_bit,pixel_representation,lossy_image_compression,private_creator,private_tag_data,[unknown],derivation_description,study_description
0,DERIVED,SECONDARY,ABDOMINAL,0001,GEMSSINGLEFRAME,GEMSMGCOUNT1,1.2.840.10008.5.1.4.1.1.6.1,1.2.840.113619.2.455.140911794567.1637692529.15,20211123,20211123,20211123,122455,122455,123529,,US,GE Healthcare,LA JOLLA CAMPUS,(),ACUS502330US7,"(A, D, M)",LOGIQE10,(),UP2101_US_1,,000000,O,999Y,502330US7,LOGIQE10:R2.5.3,-1,C1-6,574,1373257431,"[[(0018, 6012) Region Spatial Format ...",1,1,0,2,109,1057,805,527,18,3,3,0.0,0.0,0.026664,0.026664,4500,CURVED LINEAR,1.2.840.113619.2.455.140911794567.1637691895.2,1.2.840.113619.2.455.140911794567.1637691895.3,20211123.122455,1,120,,3,RGB,0,248,264,0,8,8,7,0,00,GEMS_Ultrasound_MovieGroup_001,[],IQ,SmallPreview,NaN
1,DERIVED,SECONDARY,ABDOMINAL,0001,GEMSSINGLEFRAME,GEMSMGCOUNT1,1.2.840.10008.5.1.4.1.1.6.1,1.2.840.113619.2.455.140911794567.1637692529.15,20211123,20211123,20211123,122455,122455,123529,,US,GE Healthcare,LA JOLLA CAMPUS,(),ACUS502330US7,"(A, D, M)",LOGIQE10,(),UP2101_US_1,,000000,O,999Y,502330US7,LOGIQE10:R2.5.3,-1,C1-6,574,1373257431,"[[(0018, 6012) Region Spatial Format ...",1,1,0,2,109,1057,805,527,18,3,3,0.0,0.0,0.026664,0.026664,4500,CURVED LINEAR,1.2.840.113619.2.455.140911794567.1637691895.2,1.2.840.113619.2.455.140911794567.1637691895.3,20211123.122455,1,120,,3,RGB,0,248,264,0,8,8,7,0,00,GEMS_Ultrasound_MovieGroup_001,[],IQ,SmallPreview,NaN
2,DERIVED,SECONDARY,ABDOMINAL,0001,GEMSSINGLEFRAME,GEMSMGCOUNT1,1.2.840.10008.5.1.4.1.1.6.1,1.2.840.113619.2.455.140911794567.1637692532.16,20211123,20211123,20211123,122455,122455,123532,,US,GE Healthcare,LA JOLLA CAMPUS,(),ACUS502330US7,"(A, D, M)",LOGIQE10,(),UP2101_US_1,,000000,O,999Y,502330US7,LOGIQE10:R2.5.3,-1,C1-6,574,1373257431,"[[(0018, 6012) Region Spatial Format ...",1,1,0,2,109,1057,805,527,18,3,3,0.0,0.0,0.026664,0.026664,4500,CURVED LINEAR,1.2.840.113619.2.455.140911794567.1637691895.2,1.2.840.113619.2.455.140911794567.1637691895.3,20211123.122455,1,130,,3,RGB,0,248,264,0,8,8,7,0,00,GEMS_Ultrasound_MovieGroup_001,[],IQ,SmallPreview,NaN
3,DERIVED,SECONDARY,ABDOMINAL,0001,GEMSSINGLEFRAME,GEMSMGCOUNT1,1.2.840.10008.5.1.4.1.1.6.1,1.2.840.113619.2.455.140911794567.1637692532.16,20211123,20211123,20211123,122455,122455,123532,,US,GE Healthcare,LA JOLLA CAMPUS,(),ACUS502330US7,"(A, D, M)",LOGIQE10,(),UP2101_US_1,,000000,O,999Y,502330US7,LOGIQE10:R2.5.3,-1,C1-6,574,1373257431,"[[(0018, 6012) Region Spatial Format ...",1,1,0,2,109,1057,805,527,18,3,3,0.0,0.0,0.026664,0.026664,4500,CURVED LINEAR,1.2.840.113619.2.455.140911794567.1637691895.2,1.2.840.113619.2.455.140911794567.1637691895.3,20211123.122455,1,130,,3,RGB,0,248,264,0,8,8,7,0,00,GEMS_Ultrasound_MovieGroup_001,[],IQ,SmallPreview,NaN
4,DERIVED,SECONDARY,ABDOMINAL,0001,GEMSSINGLEFRAME,GEMSMGCOUNT1,1.2.

In [28]:
df_imgs.shape

(624, 74)

In [30]:
df['study_date'] = df_imgs['study_date']

In [31]:
print(df.columns)

Index(['directory', 'rfFilename', 'nameTag', 'participantID', 'examID', 'harmonics', 'liverID', 'imageID', 'phantom', 'fMinMHz', 'fMaxMHz', 'fCenterMHz', 'ac', 'bsc', 'slope', 'intercept', 'midband', 'kappa', 'mu', 'study_date'], dtype='object')


In [33]:
col = df.pop("study_date")
df.insert(4, col.name, col)

In [34]:
print(df.columns)

Index(['directory', 'rfFilename', 'nameTag', 'participantID', 'study_date', 'examID', 'harmonics', 'liverID', 'imageID', 'phantom', 'fMinMHz', 'fMaxMHz', 'fCenterMHz', 'ac', 'bsc', 'slope', 'intercept', 'midband', 'kappa', 'mu'], dtype='object')


In [35]:
df.head(100)

,directory,rfFilename,nameTag,participantID,study_date,examID,harmonics,liverID,imageID,phantom,fMinMHz,fMaxMHz,fCenterMHz,ac,bsc,slope,intercept,midband,kappa,mu
0,/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...,IM-0001-0120_RF,UP2101_Exam1_HarmonicsON_LiverA_Image1_Phantom...,UP2101,20211123,Exam1,HarmonicsON,LiverA,Image1,PhantomContemp,2.5,3.5,3,0.794865,0.005931,3.544063,-33.064277,-22.432089,1.011658,10.977767
1,/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...,IM-0001-0120_RF,UP2101_Exam1_HarmonicsON_LiverA_Image1_Phantom...,UP2101,20211123,Exam1,HarmonicsON,LiverA,Image1,PhantomStandard,2.5,3.5,3,0.765666,0.005043,3.467101,-33.543032,-23.141727,1.011658,10.977767
2,/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...,IM-0001-0130_RF,UP2101_Exam1_HarmonicsON_LiverA_Image2_Phantom...,UP2101,20211123,Exam1,HarmonicsON,LiverA,Image2,PhantomContemp,2.5,3.5,3,0.747298,0.004098,4.670096,-38.059999,-24.049711,0.956371,10.127541
3,/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...,IM-0001-0130_RF,UP2101_Exam1_HarmonicsON_LiverA_Image2_Phantom...,UP2101,20211123,Exam1,HarmonicsON,LiverA,Image2,PhantomStandard,2.5,3.5,3,0.733266,0.004039,5.628811,-41.064438,-24.178006,0.956371,10.127541
4,/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...,IM-0001-0140_RF,UP2101_Exam1_HarmonicsON_LiverA_Image3_Phantom...,UP2101,20211123,Exam1,HarmonicsON,LiverA,Image3,PhantomContemp,2.5,3.5,3,0.792950,0.005859,3.583021,-33.241912,-22.492849,0.952656,10.071296
5,/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...,IM-0001-0140_RF,UP2101_Exam1_HarmonicsON_LiverA_Image3_Phantom...,UP2101,20211123,Exam1,HarmonicsON,LiverA,Image3,PhantomStandard,2.5,3.5,3,0.770669,0.005288,3.227513,-32.612604,-22.930065,0.952656,10.071296
6,/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...,IM-0001-0150_RF,UP2101_Exam1_HarmonicsON_LiverA_Image4_Phantom...,UP2101,20211123,Exam1,HarmonicsON,LiverA,Image4,PhantomContemp,2.5,3.5,3,0.752731,0.004220,3.411803,-34.080299,-23.844890,0.959822,9.872428
7,/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...,IM-0001-0150_RF,UP2101_Exam1_HarmonicsON_LiverA_Image4_Phantom...,UP2101,20211123,Exam1,HarmonicsON,LiverA,Image4,PhantomStandard,2.5,3.5,3,0.747583,0.004364,2.770395,-31.996847,-23.685661,0.959822,9.872428
8,/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...,IM-0001-0160_RF,UP2101_Exam1_HarmonicsON_LiverA_Image5_Phantom...,UP2101,20211123,Exam1,HarmonicsON,LiverA,Image5,PhantomContemp,2.5,3.5,3,0.816215,0.006344,3.158301,-31.560009,-22.085106,0.945820,10.405085
9,/data/QUS/UCSDLiver/Peds_Pilot_Data/UIUC_Clini...,IM-0001-0160_RF,UP2101_Exam1_HarmonicsON_LiverA_Image5_Phantom...,UP2101,20211123,Exam1,HarmonicsON,LiverA,Image5,PhantomStandard,2.5,3.5,3,0.810027,0.006540,2.553855,-29.605438,-21.943874,0.945820,10.405085


In [42]:
df.to_excel('Ped_liver_QUS2.xlsx', index= False, sheet_name= 'QUS')